In [8]:
!pip install BCEmbedding==0.1.3

In [7]:
!bash
!setproxy

se) ]0;liangzhu@user-SYS-420GP-TNAR: ~/code/dev/learn-AI/LLM/4.RAG/3.framework/1.QAnythingliangzhu@user-SYS-420GP-TNAR:~/code/dev/learn-AI/LLM/4.RAG/3.framework/1.QAnything$ ^C

(base) ]0;liangzhu@user-SYS-420GP-TNAR: ~/code/dev/learn-AI/LLM/4.RAG/3.framework/1.QAnythingliangzhu@user-SYS-420GP-TNAR:~/code/dev/learn-AI/LLM/4.RAG/3.framework/1.QAnything$ /bin/sh: 1: setproxy: not found


In [9]:
# 我们在`BCEmbedding`中提供langchain直接集成的接口。
from BCEmbedding.tools.langchain import BCERerank

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.retrievers import ContextualCompressionRetriever


# init embedding model
embedding_model_name = 'maidalun1020/bce-embedding-base_v1'
embedding_model_kwargs = {'device': 'cuda:0'}
embedding_encode_kwargs = {'batch_size': 32, 'normalize_embeddings': True, 'show_progress_bar': False}

embed_model = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=embedding_model_kwargs,
  encode_kwargs=embedding_encode_kwargs
)

reranker_args = {'model': 'maidalun1020/bce-reranker-base_v1', 'top_n': 5, 'device': 'cuda:1'}
reranker = BCERerank(**reranker_args)

# init documents
documents = PyPDFLoader("./data/高速公路养护管理办法.pdf").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# example 1. retrieval with embedding and reranker
retriever = FAISS.from_documents(texts, embed_model, distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT).as_retriever(search_type="similarity", search_kwargs={"score_threshold": 0.3, "k": 10})

compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker, base_retriever=retriever
)

response = compression_retriever.get_relevant_documents("What is Llama 2?")

04/08/2024 15:50:53 - [INFO] -sentence_transformers.SentenceTransformer->>>    Load pretrained SentenceTransformer: maidalun1020/bce-embedding-base_v1
04/08/2024 15:51:03 - [WARNING] -sentence_transformers.SentenceTransformer->>>    No sentence-transformers model found with name maidalun1020/bce-embedding-base_v1. Creating a new one with MEAN pooling.


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like maidalun1020/bce-embedding-base_v1 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.